In [5]:
import os
import fnmatch
import nibabel as nib
import numpy as np
import pandas as pd

In [6]:
def calculate_tumor_volumn(segmentation_path, tumor_mask):
    segmentation_mask = nib.load(segmentation_path).get_fdata()
    voxel_dimensions = nib.load(segmentation_path).header.get_zooms()
    voxel_volume = np.prod(voxel_dimensions)  # in mm³
    num_tumor_voxels = np.sum(tumor_mask)
    tumor_volume = num_tumor_voxels * voxel_volume  # in mm³
    return tumor_volume

In [7]:
def list_all_files(root_dir, prefix):
    matching_folders = []
    for root, dirs, files in os.walk(root_dir):
        for dir_name in dirs:
            if fnmatch.fnmatch(dir_name, f'{prefix}*'):
                matching_folders.append(f'dataset/BraTS2021_Training_Data/{dir_name}/{dir_name}_seg.nii.gz')
    return matching_folders

# Example usage:
root_directory = 'dataset/BraTS2021_Training_Data'
prefix = 'BraTS2021_'
folders = list_all_files(root_directory, prefix)

In [8]:
result = { 'file': [], 'volumn (mm3)': []}

result_df = pd.DataFrame(result)

for folder in folders:
    segmentation_mask = nib.load(folder).get_fdata()
    tumor_mask = segmentation_mask == 4
    volumn = calculate_tumor_volumn(folder, tumor_mask)
    data = { 'file': folder, 'volumn (mm3)': volumn }

    result_df.loc[len(result_df)] = data

result_df.to_csv('4-tumor-volumn.csv')